In [20]:
import pandas as pd
import numpy as np

In [21]:
# set paths to csv files
accounts = './data/accounts.csv'
playbacks = './data/playbacks.csv'
subscriptions = './data/subscriptions.csv'
vouchers = './data/promo_vouchers.csv'

In [22]:
# Import the get_engine function from our sql_functions.
from sql_functions import get_engine #adjust this as necessary to match your sql_functions.py connection methods

# ACCOUNTS

In [4]:
# Read accounts (21.10.2020 - 01.10.2022)
df_accounts = pd.read_csv(accounts)
# set column names to lowercase
df_accounts.columns = df_accounts.columns.str.lower()

## Postal Code

In [5]:
# remove non numeric characters
df_accounts['postal_code_clean'] = df_accounts['postal_code'].str.replace('-', '')
df_accounts['postal_code_clean'] = df_accounts['postal_code_clean'].str.extract('(\d+)')
# fill null-values with 0
df_accounts['postal_code_clean'].fillna(0, inplace=True)

In [6]:
# change data type to integer
df_accounts['postal_code_clean'] = df_accounts['postal_code_clean'].astype(int)

## City

### Mapping plz_files to accounts table for further geographical information

In [7]:
# set file paths
plz_ch = './data/plz_verzeichnis_ch.csv'
plz_kanton = './data/plz_kantone_ch.csv'
plz_de = './data/plz_verzeichnis_de.csv'
plz_at = './data/plz_verzeichnis_at.csv'
# read csv files
df_plz_ch = pd.read_csv(plz_ch, sep=';')
df_plz_kanton = pd.read_csv(plz_kanton, sep=';')
df_plz_de = pd.read_csv(plz_de, sep=',')
df_plz_at = pd.read_csv(plz_at, sep=';')
# set column names to lowercase
df_plz_ch.columns = df_plz_ch.columns.str.lower()
df_plz_kanton.columns = df_plz_kanton.columns.str.lower()
df_plz_de.columns = df_plz_de.columns.str.lower()
df_plz_at.columns = df_plz_at.columns.str.lower()


In [8]:
# clean plz_kanton
# only keep relevant columns, rename
df_plz_kanton = df_plz_kanton[['postleitzahl / code postal / codice postale', 'ort / ville / città', 'kanton']]
df_plz_kanton.rename(columns = {'postleitzahl / code postal / codice postale':'postal_code', 'ort / ville / città':'city', 'kanton':'state'}, inplace = True)
df_plz_kanton.drop_duplicates(inplace = True)
# add country_code for differentiation
df_plz_kanton['country_code'] = 'CH'

In [9]:
# clean plz_de
# only keep relevant columns, rename, drop duplicates
df_plz_de = df_plz_de[['plz', 'ort', 'bundesland']]
df_plz_de.rename(columns = {'plz':'postal_code', 'ort':'city', 'bundesland':'state'}, inplace = True)
df_plz_de.drop_duplicates(inplace = True)
# add country_code for differentiation
df_plz_de['country_code'] = 'DE'

In [10]:
# clean plz_at
# only keep relevant columns, rename, drop duplicates
df_plz_at = df_plz_at[['plz', 'ort', 'bundesland']]
df_plz_at.rename(columns = {'plz':'postal_code', 'ort':'city', 'bundesland':'state'}, inplace = True)
df_plz_at.drop_duplicates(inplace = True)
# add country_code for differentiation
df_plz_at['country_code'] = 'AT'

In [11]:
# unify for merging, check shape
df_plz_all = pd.concat([df_plz_kanton, df_plz_de, df_plz_at])
# dropping plz duplicates with multiple city, keeping the first entry each
df_plz_all = df_plz_all.groupby(['postal_code'])['city', 'state', 'country_code'].first().reset_index()

/var/folders/0j/szrk2v_52_bgyh57zj48fk400000gn/T/ipykernel_98153/1600884800.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_plz_all = df_plz_all.groupby(['postal_code'])['city', 'state', 'country_code'].first().reset_index()


In [12]:
# merge city and state information to accounts table on plz and country code
df_accounts = pd.merge(df_accounts, df_plz_all, left_on=['postal_code_clean', 'country_code'], right_on=['postal_code', 'country_code'], how='left')

In [13]:
# clean plz_a for language information
# only keep relevant columns, rename, drop duplicates
df_plz_ch = df_plz_ch[['postleitzahl', 'sprachcode']]
df_plz_ch.rename(columns = {'postleitzahl':'postal_code'}, inplace = True)
# add country_code for differentiation
df_plz_ch['country_code'] = 'CH'
#drop duplicates for dual-language-cities and keep first entry
df_plz_ch = df_plz_ch.groupby(['postal_code'])['sprachcode', 'country_code'].first().reset_index()

'''
Mapping of the language code:
#1 = German  
#2 = French  
#3 = Italian 
'''

/var/folders/0j/szrk2v_52_bgyh57zj48fk400000gn/T/ipykernel_98153/101013062.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_plz_ch = df_plz_ch.groupby(['postal_code'])['sprachcode', 'country_code'].first().reset_index()


,postal_code,sprachcode,country_code
0,1000,2,CH
1,1001,2,CH
2,1002,2,CH
3,1003,2,CH
4,1004,2,CH


(3488, 3)

'\nMapping of the language code:\n#1 = German  \n#2 = French  \n#3 = Italian \n'

In [14]:
# merge language code to accounts table
df_accounts = pd.merge(df_accounts, df_plz_ch, left_on=['postal_code_clean', 'country_code'], right_on=['postal_code', 'country_code'], how='left')

In [15]:
#drop duplicate postal code columns
df_accounts = df_accounts.drop(['postal_code_x', 'postal_code_y', 'postal_code'], axis=1)
#rename original postal code column
df_accounts.rename(columns = {'postal_code_x':'postal_code_original', 'city_x':'city_original', 'city_y':'city_clean', 'sprachcode': 'language_code'}, inplace = True)

## Country_Code

### Add country name and region information

In [16]:
# add country information
country= './data/country_code.csv'
df_country = pd.read_csv(country)
# make column names lowercase
df_country.columns = df_country.columns.str.lower()

In [17]:
#only keep relevant columns, rename
df_country = df_country[['name', 'alpha-2', 'region', 'sub-region']]
df_country.rename(columns = {'alpha-2':'country_code', 'name':'country_name', 'sub-region':'sub_region'}, inplace = True)

In [18]:
#merge to accounts_new table
df_accounts = pd.merge(df_accounts, df_country, on='country_code', how='left')

## Language

In [19]:
#fill null values
df_accounts['language'].fillna('na', inplace=True)

# PLAYBACKS

In [23]:
# Read playbacks
df_playbacks = pd.read_csv(playbacks)
# set column names to lowercase
df_playbacks.columns = df_playbacks.columns.str.lower()


## Changing datatypes and adding column playback_ID

In [24]:
# changing datatype to datetime
df_playbacks['date_start'] = pd.to_datetime(df_playbacks['date_start'])

In [25]:
# adding playback_ID column and adding incremental nr as playback_ID to every row starting from the total count of rows, descending
df_playbacks.insert(0, 'playback_ID', range(len(df_playbacks), 0, -1))

## Add category according to user_agent

In [26]:
#adding the infos about device of playback and app users
df_playbacks.loc[df_playbacks['user_agent'].str.contains('Windows|Macintosh|TV|Linux|Darwin|CrOS|PlayStation|FreeBSD'), 'device'] = 'desktop'
df_playbacks.loc[df_playbacks['user_agent'].str.contains('Android|iOS|iPhone|iPad'), 'device'] = 'mobile'
df_playbacks.loc[df_playbacks['user_agent'].str.contains('filmingo'), 'app_user'] = 'yes'
df_playbacks.loc[~df_playbacks['user_agent'].str.contains('filmingo'), 'app_user'] = 'no'
df_playbacks.groupby('device').count()

,playback_ID,subscription_key,account_key,movie_id,date_start,playback_time,user_agent,ip_hash,app_user
device,,,,,,,,,
desktop,116769,116769,116769,116769,116769,116769,116769,116769,116769
mobile,22800,22800,22800,22800,22800,22800,22800,22800,22800


# SUBSCRIPTIONS

In [8]:
# Read subscriptions
df_subscriptions = pd.read_csv(subscriptions)
# set column names to lowercase
df_subscriptions.columns = df_subscriptions.columns.str.lower()

In [9]:
#set data types for subscription dates to datetime
df_subscriptions['subscription_start'] = pd.to_datetime(df_subscriptions['subscription_start'])
df_subscriptions['subscription_end'] = pd.to_datetime(df_subscriptions['subscription_end'])

In [10]:
# drop "wrong" subscription type line (FULLACCESS - unknown type to us - only one line therefore decided to drop)
df_subscriptions.drop(df_subscriptions[(df_subscriptions['subscription_type'] == 'FULLACCESS')].index, inplace = True)

In [11]:
#create new column and calculate subscription duration
df_subscriptions['subscription_months_raw'] = ((df_subscriptions.subscription_end) - df_subscriptions.subscription_start)/np.timedelta64(1, 'M')

In [12]:
#create new column for rounded subscription months for easier further processing
#generally round up from 0.1 to be able to allow some discrepancies due to day to day calculation of subscription duration (deduct 0.1 to be able to use .ceil)
df_subscriptions['subscription_months'] = df_subscriptions['subscription_months_raw'] - 0.1
df_subscriptions['subscription_months'] = df_subscriptions['subscription_months'].apply(np.ceil)

In [13]:
# Create two columns for chf and eur based on the subscription_type and prices from the filmingo website

# create a list of the conditions
conditions = [
    ((df_subscriptions['subscription_type'] == 'BASIC') & (df_subscriptions['subscription_monthly'] == 0)),
    ((df_subscriptions['subscription_type'] == 'BASIC') & (df_subscriptions['subscription_monthly'] == 1)),
    ((df_subscriptions['subscription_type'] == 'STANDARD') & (df_subscriptions['subscription_monthly'] == 0)),
    ((df_subscriptions['subscription_type'] == 'STANDARD') & (df_subscriptions['subscription_monthly'] == 1)),
    ((df_subscriptions['subscription_type'] == 'PATRON') & (df_subscriptions['subscription_monthly'] == 0))

]

# create a list of the values we want to assign for each condition
values_chf = ['90.0', '9.0', '150.0', '15.0', '240.0']
values_eur = ['75.0', '7.5', '125.0', '12.5', '200.0']

# create a new column and use np.select to assign values to it using our lists as arguments
df_subscriptions['price_chf'] = np.select(conditions, values_chf)
df_subscriptions['price_eur'] = np.select(conditions, values_eur)

#change datatype into float for further calculation
df_subscriptions['price_chf'] = df_subscriptions.price_chf.astype('float')
df_subscriptions['price_eur'] = df_subscriptions.price_eur.astype('float')

# decided to use these prices for all subscriptions regardless if they might have a different prices in the list (possibly due to discounts, total lines of abnormal prices: 39) or are gifted subscription (price: NaN, total lines 1.636)


In [ ]:
#calculate total price per subscription (price / 12 * subscription months)
df_subscriptions['total_price_chf'] = df_subscriptions['price_chf'] / 12 * df_subscriptions['subscription_months']
df_subscriptions['total_price_eur'] = df_subscriptions['price_eur'] / 12 * df_subscriptions['subscription_months']


# conditional calculation for exceptions:

# if the subscription is monthly only calculate price * months
df_subscriptions.loc[(df_subscriptions['subscription_monthly'] == 1), 'total_price_chf'] = (df_subscriptions['price_chf'] * df_subscriptions['subscription_months'])
df_subscriptions.loc[(df_subscriptions['subscription_monthly'] == 1), 'total_price_eur'] = (df_subscriptions['price_eur'] * df_subscriptions['subscription_months'])

# if the subscription is gifted & 6 months long, a different price is applicable (there is only a 6 month subscription available for gifted subscriptions)
df_subscriptions.loc[((df_subscriptions['gift_subscription'] == True) & (df_subscriptions['subscription_months'] == 6)), 'total_price_chf'] = '49'
df_subscriptions.loc[((df_subscriptions['gift_subscription'] == True) & (df_subscriptions['subscription_months'] == 6)), 'total_price_eur'] = '41'

# VOUCHERS

In [ ]:
# Read promo_couchers
df_vouchers = pd.read_csv(vouchers, sep=';')
# set column names to lowercase
df_vouchers.columns = df_vouchers.columns.str.lower()

In [ ]:
# changing datatype to datetime
df_vouchers['creationdate'] = pd.to_datetime(df_vouchers['creationdate'])
df_vouchers['expirationdate'] = pd.to_datetime(df_vouchers['expirationdate'])

In [ ]:
#cut off time of creation -> time not relevant? 
#df_vouchers['creationdate'] = df_vouchers['creationdate'].dt.date -> does not work very well as it returns an object
df_vouchers['creationdate'] = df_vouchers['creationdate'].dt.normalize()

In [ ]:
# adding playback_ID column and adding incremental nr as playback_ID to every row starting from 1, ascending
df_vouchers.insert(0, 'voucher_ID', range(1, 1 + len(df_vouchers)))

In [ ]:
#rename columns
df_vouchers.rename(columns = {'account_key':'account_key_sender', 'email_hash':'email_hash_receiver', 'voucherused': 'voucher_used', 'creationdate': 'creation_date', 'expirationdate': 'expiration_date'}, inplace = True)


# PUSH DATA TO SQL SERVER

In [27]:
schema = 'capstone_filmingo' # UPDATE 'TABLE_SCHEMA' based on schema used in class 
engine = get_engine() # assign engine to be able to query against the database

## Accounts

In [23]:
# Specify which table within your database you want to push your data to. Give your table an unambiguous name.
# Example: flights_sp for Sina's flights table, flights_groupname or similar
table_name = 'accounts'
# If the specified table doesn't exist yet, it will be created
# With 'replace', your data will be replaced if the table already exists.
# This may take some time ...

# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        df_accounts.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The accounts table was imported successfully.


## Subscriptions

In [ ]:
# Specify which table within your database you want to push your data to. Give your table an unambiguous name.
# Example: flights_sp for Sina's flights table, flights_groupname or similar
table_name = 'subscriptions'
# If the specified table doesn't exist yet, it will be created
# With 'replace', your data will be replaced if the table already exists.
# This may take some time ...

# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        df_subscriptions.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

## Playback

In [28]:
# Specify which table within your database you want to push your data to. Give your table an unambiguous name.
# Example: flights_sp for Sina's flights table, flights_groupname or similar
table_name = 'playbacks'
# If the specified table doesn't exist yet, it will be created
# With 'replace', your data will be replaced if the table already exists.
# This may take some time ...

# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        df_playbacks.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

## Promo Voucher

In [ ]:
# Specify which table within your database you want to push your data to. Give your table an unambiguous name.
# Example: flights_sp for Sina's flights table, flights_groupname or similar
table_name = 'vouchers'
# If the specified table doesn't exist yet, it will be created
# With 'replace', your data will be replaced if the table already exists.
# This may take some time ...

# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        df_vouchers.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

In [ ]:
# Just to be sure: Check if the number of rows match
table_name_sql = f'''SELECT count(*) 
                    FROM {schema}.{table_name}
                    '''
engine.execute(table_name_sql).fetchall()[0][0] == df_vouchers.shape[0]